# Template para verificar desempenho do Modelo - AutoML

#### Importando Biblioteca de Funções Turing Lab e Turing IA

In [1]:
%run -i '/home/.Turing/TuringCredentialsAccess.py3'
%run -i '/home/.Turing/TuringLabFunctions.py3'

-------------------- LabTuring ---------------------------
----------- Exploração de Dados e Modelagem --------------
------ Funções carregadas em memória com sucesso ---------
--------- Data da última atualização: --------------------
--------------- 10/11/2019 -------------------------------
----------------------------------------------------------


#### Mapear bibliotecas Python necessárias

In [2]:
import pandas as pd
import numpy as np

In [3]:
#---------- Definição dos Parâmetros do projeto ---------------------------------
nm_bucket = 'turing-bkt-treinamentos-prod'
id_empresa = '1022'

# Id do projeto parametrizado no Turing (o mesmo valor dado ao campo Id Projeto do Timeline)
id_projeto = '1006'

# Nome da variável resposta (quando houve)
targetname = 'TARGET'

# Delimitador da tabela 
abt_delimiter = ','

In [4]:
################### Não alterar este parágrafo #####################################
#---------- Gerando Credenciais para Acesso Seguro ---------------------------------
id_modelo = id_projeto
schema = 'EMP_'+ id_empresa +'_Proj_'+ id_projeto
S3fs,S3session,S3client,S3resource = TuringUsersCredentialsControl(id_empresa)
schema_rds = 'EMP_'+id_empresa+'_Proj_'+id_projeto
mysq_conn,engine = TuringRDSManagementConn(id_empresa,schema_rds,turing_adm = 'N')
path_s3_wfile = 's3://'+nm_bucket+'/Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
s3_path = path_s3_wfile
####################################################################################

Acesso RDS Turing Treinamentos


#### Leitura da tabela desenvolvimento (holdout) escorada pelo Turing 

In [5]:
nm_file_desenv_holdout = 'scored_best_model_desenv.csv'
path_ = 'Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
path_file = path_+str(nm_file_desenv_holdout)
desenv_df_ho = TuringReadS3CSVFile(nm_bucket,path_file,sep=abt_delimiter)

desenv_df_ho.rename( columns={'Unnamed: 0':'Id'}, inplace=True )
desenv_df_ho.shape

(167900, 24)

In [6]:
desenv_df_ho.head()

,Id,MAX_AMT_CREDIT_SUM,DAYS_EMPLOYED,EXT_SOURCE_2,MAX_DAYS_CREDIT,AVG_AMT_CREDIT_SUM_OVERDUE,OWN_CAR_AGE,EXT_SOURCE_3,EXT_SOURCE_1,AVG_AMT_CREDIT_SUM_DEBT,...,MAX_AMT_CREDIT_SUM_OVERDUE,AMT_ANNUITY,FLAG_DOCUMENT_3,AMT_CREDIT,AVG_AMT_CREDIT_Microloan,AMT_GOODS_PRICE,P_Target_0,P_Target_1,C_Target,target_t_bin
0,1790,-0.487279,-0.457455,-3.491085e-15,0.613709,-0.009714,0.135076,0.271335,1.572785e+00,-0.310873,...,-0.014673,-1.059198,0.639141,-0.930905,-0.029989,-0.849993,0.958541,0.041459,0,0
1,142827,-0.462161,-0.452611,1.422960e+00,-0.092169,-0.009714,-0.295881,0.014703,1.274772e-14,-0.198025,...,-0.014673,0.193429,0.639141,-0.785075,-0.029989,-0.764537,0.968707,0.031293,0,0
2,116554,0.485445,-0.470393,8.631666e-01,-0.241412,-0.009714,0.000000,0.601282,-4.613518e-02,0.134185,...,-0.014673,-1.251384,-1.564600,-1.042919,-0.029989,-0.972072,0.982291,0.017709,0,0
3,18449,-0.172171,-0.501020,-3.633448e-02,0.214384,-0.009714,0.000000,0.839871,-2.873602e-01,-0.041509,...,-0.014673,-0.758480,0.639141,-0.930905,-0.029989,-0.849993,0.965812,0.034188,0,0
4,38196,-0.453996,-0.452526,9.632383e-01,-0.554016,-0.009714,0.000000,1.056560,1.274772e-14,-0.333474,...,-0.014673,-1.406873,0.639141,-1.154933,-0.029989,-1.094152,0.972776,0.027224,0,0


#### Leitura da tabela validaçao (holdout) escorada pelo Turing 

In [7]:
nm_file_valid_holdout = 'scored_best_model_valid.csv'
path_ = 'Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
path_file = path_+str(nm_file_valid_holdout)
valid_df_ho = TuringReadS3CSVFile(nm_bucket,path_file,sep=abt_delimiter)

valid_df_ho.rename( columns={'Unnamed: 0':'Id'}, inplace=True )
valid_df_ho.shape

(47357, 24)

In [8]:
valid_df_ho.head()

,Id,MAX_AMT_CREDIT_SUM,DAYS_EMPLOYED,EXT_SOURCE_2,MAX_DAYS_CREDIT,AVG_AMT_CREDIT_SUM_OVERDUE,OWN_CAR_AGE,EXT_SOURCE_3,EXT_SOURCE_1,AVG_AMT_CREDIT_SUM_DEBT,...,MAX_AMT_CREDIT_SUM_OVERDUE,AMT_ANNUITY,FLAG_DOCUMENT_3,AMT_CREDIT,AVG_AMT_CREDIT_Microloan,AMT_GOODS_PRICE,P_Target_0,P_Target_1,C_Target,target_t_bin
0,54907,1.813890e-01,-0.482042,1.375146,2.547197e-01,-9.713821e-03,7.461355,-8.619551e-01,-1.013052e+00,-2.912181e-01,...,-1.467338e-02,-0.594283,0.639141,-0.647555,-2.998867e-02,-0.666874,0.928656,0.071344,0,0
1,129635,-4.340300e-01,-0.464302,-0.614790,-7.859473e-01,-9.713821e-03,1.284297,6.595953e-01,1.274772e-14,-3.334743e-01,...,-1.467338e-02,-1.251384,-1.564600,-1.042919,-2.998867e-02,-0.972072,0.951507,0.048493,0,0
2,58415,-5.316429e-16,-0.514177,1.294220,-3.439248e-16,-5.436170e-18,0.000000,3.814783e-15,2.706518e+00,-1.812817e-15,...,4.501392e-18,-0.432573,-1.564600,-0.586922,3.586454e-17,-0.569210,0.984494,0.015506,0,0
3,189828,-3.251580e-01,2.135083,0.385676,-6.467884e-01,-9.713821e-03,0.000000,3.814783e-15,2.892588e+00,-3.334743e-01,...,-1.467338e-02,-0.123459,-1.564600,-0.856976,-2.998867e-02,-0.849993,0.962664,0.037336,0,0
4,190654,1.408817e-01,-0.458743,0.565544,5.632895e-01,-9.713821e-03,0.000000,-1.293939e+00,1.035201e+00,3.539486e-02,...,-1.467338e-02,0.090495,-1.564600,-0.258822,-2.998867e-02,-0.117516,0.917996,0.082004,0,0


#### Calculando KS - Desenvolvimento

In [9]:
KS_df,KS_value_d = TuringEvaluateModels(desenv_df_ho,'P_Target_0','target_t_bin')
KS_df.head(10)

,min_scr,max_scr,bads,goods,total,odds,bad_rate,ks,max_ks
0,0.315813,0.819829,9245,7545,16790,0.82,55.06%,63.71,
1,0.819830,0.878522,3421,13369,16790,3.91,20.38%,80.44,<== KS ==
2,0.878524,0.910291,737,16053,16790,21.78,4.39%,75.51,
3,0.910291,0.931076,70,16720,16790,238.86,0.42%,65.20,
4,0.931076,0.945876,3,16787,16790,5595.67,0.02%,54.36,
5,0.945878,0.957217,1,16789,16790,16789.00,0.01%,43.49,
6,0.957218,0.966433,0,16790,16790,inf,0.00%,32.62,
7,0.966433,0.974196,0,16790,16790,inf,0.00%,21.75,
8,0.974196,0.981638,0,16790,16790,inf,0.00%,10.87,
9,0.981638,0.998741,0,16790,16790,inf,0.00%,0.00,


#### Calculando KS - Validação

In [10]:
KS_df,KS_value_v = TuringEvaluateModels(valid_df_ho,'P_Target_0','target_t_bin')
KS_df.head(10)

,min_scr,max_scr,bads,goods,total,odds,bad_rate,ks,max_ks
0,0.428619,0.827473,1237,3499,4736,2.83,26.12%,24.35,
1,0.827535,0.878753,713,4023,4736,5.64,15.05%,33.78,
2,0.878781,0.907194,479,4256,4735,8.89,10.12%,36.55,<== KS ==
3,0.907199,0.926658,352,4384,4736,12.45,7.43%,35.70,
4,0.926658,0.941740,303,4433,4736,14.63,6.40%,33.45,
5,0.941741,0.953371,233,4502,4735,19.32,4.92%,29.21,
6,0.953376,0.963251,179,4557,4736,25.46,3.78%,23.43,
7,0.963253,0.971501,145,4590,4735,31.66,3.06%,16.69,
8,0.971503,0.979604,104,4632,4736,44.54,2.20%,8.77,
9,0.979606,0.997584,74,4662,4736,63.00,1.56%,0.00,


#### Calculando Gini - Desenvolvimento

In [11]:
gini_desenv = TuringGini(desenv_df_ho,'target_t_bin','P_Target_1')
gini_desenv*100

91.75999999999999

#### Calculando Gini - Validação

In [12]:
gini_valid = TuringGini(valid_df_ho,'target_t_bin','P_Target_1')
gini_valid*100

49.24

#### Consolidando Metricas 

In [13]:
metricas = [['Desenv', KS_value_d,gini_desenv], ['Valid', KS_value_v,gini_valid]]
df = pd.DataFrame(metricas, columns = ['ABT','KS', 'Gini']) 
df

,ABT,KS,Gini
0,Desenv,80.44,0.9176
1,Valid,36.55,0.4924
